In [1]:
import requests
import json

url = "https://api.bugguide.net/taxonomy/Blattodea.txt"
response = requests.get(url)

# Convert the response to a Python dictionary (if it's JSON)
try:
    data = response.json()
    # Pretty print the JSON data
    print(json.dumps(data, indent=4))
except ValueError:
    # If response is not in JSON format
    print(response.text)


Genus	Species	Family	Order	BugGuideID	URL	Common Name
Cryptocercus	punctulatus	Cryptocercidae	Blattodea	574	https://bugguide.net/node/view/574	Brown-hooded Cockroach
Pseudomops	septentrionalis	Ectobiidae	Blattodea	7856	https://bugguide.net/node/view/7856	Pale Bordered Field Cockroach
Pycnoscelus	surinamensis	Blaberidae	Blattodea	16867	https://bugguide.net/node/view/16867	Surinam Cockroach
Ectobius	lapponicus	Ectobiidae	Blattodea	26906	https://bugguide.net/node/view/26906	Dusky Cockroach
Periplaneta	australasiae	Blattidae	Blattodea	31621	https://bugguide.net/node/view/31621	Australian Cockroach
Periplaneta	americana	Blattidae	Blattodea	31622	https://bugguide.net/node/view/31622	American Cockroach
Panchlora	nivea	Blaberidae	Blattodea	31689	https://bugguide.net/node/view/31689	Green Banana Cockroach
Eurycotis	floridana	Blattidae	Blattodea	33678	https://bugguide.net/node/view/33678	Florida Woods Cockroach
Blattella	asahinai	Ectobiidae	Blattodea	33854	https://bugguide.net/node/view/33854	As

In [11]:
import psycopg2
from db_config import db_credentials

u, p, h, port, db = db_credentials.values()

def connect_to_postgresql():
    try:
        connection = psycopg2.connect(
            user=u,
            password=p,
            host=h,
            port=port,
            database=db
        )
        
        # Check if the connection is successful
        if connection:
            print("Successfully Connected")
        return connection

    except Exception as error:
        print(f"Error while connecting to PostgreSQL: {error}")
        return None

# Test the function
connection = connect_to_postgresql()

# Close the connection after use
if connection:
    connection.close()

Successfully Connected


In [12]:
import psycopg2
from db_config import db_credentials

u, p, h, port, db = db_credentials.values()

def get_row_count(table_name):
    try:
        connection = psycopg2.connect(
            user=u,
            password=p,
            host=h,
            port=port,
            database=db
        )

        cursor = connection.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM \"{table_name}\"")  # Table name enclosed in double quotes
        count = cursor.fetchone()[0]

        print(f"There are {count} rows in the '{table_name}' table.")

        # Close cursor and connection
        cursor.close()
        connection.close()

        return count

    except Exception as error:
        print(f"Error: {error}")
        return None

# Replace 'your_table_name' with the name of your table
row_count = get_row_count('bugguide')


There are 40511 rows in the 'bugguide' table.


In [13]:
import psycopg2
from db_config import db_credentials

u, p, h, port, db = db_credentials.values()

def list_and_count_columns(table_name):
    try:
        connection = psycopg2.connect(
            user=u,
            password=p,
            host=h,
            port=port,
            database=db
        )

        cursor = connection.cursor()

        # Query to get column names
        cursor.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}'")
        columns = [row[0] for row in cursor.fetchall()]

        # Counting the columns
        num_columns = len(columns)

        print(f"The table '{table_name}' has {num_columns} columns.")
        print(f"Columns: {', '.join(columns)}")

        # Close cursor and connection
        cursor.close()
        connection.close()

        return num_columns, columns

    except Exception as error:
        print(f"Error: {error}")
        return None, []

# Replace 'your_table_name' with the name of your table
num_columns, columns_list = list_and_count_columns('bugguide')


The table 'bugguide' has 8 columns.
Columns: Number, Genus, Species, Family, Order, BugGuideID, URL, CommonName


In [14]:
import pandas as pd
from sqlalchemy import create_engine
from db_config import db_credentials

# Database connection setup
u, p, h, port, db = db_credentials.values()
engine_url = f"postgresql+psycopg2://{u}:{p}@{h}:{port}/{db}"
engine = create_engine(engine_url)

datasets = [
    # 'service_requests',
    'bugguide',
]

for dataset in datasets:
    sql_query = f'SELECT * FROM "{dataset}"'
    df = pd.read_sql(sql_query, engine)

df

,Number,Genus,Species,Family,Order,BugGuideID,URL,CommonName
0,1,Cryptocercus,punctulatus,Cryptocercidae,Blattodea,574,https://bugguide.net/node/view/574,Brown-hooded Cockroach
1,2,Pseudomops,septentrionalis,Ectobiidae,Blattodea,7856,https://bugguide.net/node/view/7856,Pale Bordered Field Cockroach
2,3,Pycnoscelus,surinamensis,Blaberidae,Blattodea,16867,https://bugguide.net/node/view/16867,Surinam Cockroach
3,4,Ectobius,lapponicus,Ectobiidae,Blattodea,26906,https://bugguide.net/node/view/26906,Dusky Cockroach
4,5,Periplaneta,australasiae,Blattidae,Blattodea,31621,https://bugguide.net/node/view/31621,Australian Cockroach
...,...,...,...,...,...,...,...,...
40506,40507,Mirolepisma,deserticola,Lepismatidae,Zygentoma,389939,https://bugguide.net/node/view/389939,Mirolepisma deserticola
40507,40508,Nicoletia,wheeleri,Nicoletiidae,Zygentoma,1088532,https://bugguide.net/node/view/1088532,Nicoletia wheeleri
40508,40509,Tricholepidion,gertschi,Lepidotrichidae,Zygentoma,1264432,https://bugguide.net/node/view/1264432,Relict Silverfish
40509,40510,Leucolepisma,arenarium,Lepismatidae,Zygentoma,1362001,https://bugguide.net/node/view/1362001,Leucolepisma arenarium
